In [ ]:
import json 

with open(f"../assets/corpora.json",'r') as file: 
    corpora = json.load(file)

In [ ]:
era_proportion = {}
sample_size = 200
smooth = 0
for era, corpus in corpora.items(): 
    count = sum([len(_) for _ in corpus.values()])
    era_proportion[era] = count 
total = sum(era_proportion.values()) + len(era_proportion) * smooth
era_proportion = {k: ((v+smooth)/total)*sample_size for k,v in era_proportion.items()}
era_proportion, sum(era_proportion.values())

In [ ]:
rounded = {k: round(v) for k,v in era_proportion.items()}
rounded['pre-Elizabethan'] += 1 
rounded, sum(rounded.values())

# Find missing sermons and remove duplicate adorned files 

In [16]:
import pandas as pd 
tcpIDs = pd.read_csv(f"../assets/sermons.csv")['id']
tcpIDs = list(tcpIDs)
tcpIDs.extend(pd.read_csv("../assets/sermons_missing.csv")['id'])
len(tcpIDs)

5858

In [1]:
import os 
adorn_folder = "../assets/adorned"

In [2]:
import sys
sys.path.append('../')
from lib.extract import * 

In [25]:
adorned = [file.split(".")[0] for file in os.listdir(adorn_folder)]
missing = []
for tcpID in tcpIDs: 
    if tcpID not in adorned: 
        missing.append(tcpID)
print(len(missing))

40


In [40]:
from bs4 import BeautifulSoup, SoupStrainer
import re,sys,os
sys.path.append('../') 
TCP = '/Users/amycweng/DH/TCP'

def findTextTCP(id):
    if re.match('B1|B4',id[0:2]):
        path = f'{TCP}/P2{id[0:2]}/{id}.P4.xml'
    else: 
        if f'{id}.P4.xml' in os.listdir(f'{TCP}/P1{id[0:2]}'):
            path = f'{TCP}/P1{id[0:2]}/{id}.P4.xml'
        elif f'{id}.P4.xml' in os.listdir(f'{TCP}/P2{id[0:2]}'): 
            path = f'{TCP}/P2{id[0:2]}/{id}.P4.xml'
    return path 

def extract(tcpID, filepath):
    # read the input XML file 
    with open(filepath,'r') as file: 
        data = file.read()
    # use soupstrainer to only parse the main body
    tag = SoupStrainer("DIV1")
    soup = BeautifulSoup(data,features="xml",parse_only=tag)
    
    contents = soup.find_all(['DIV1'])
    wanted_types = ["sermon","part","text","treatise","discourse","appendix", "funeral sermon","body of text","verse",
                "biblical commentary","treatise","tract","doctrine",
                "lecture", "book","conclusion","section","commentary","chapter",
                "panegyric","funeral speech","arguments","homily","colophon","polemic",
                "scaffold speech","speech","lamentation","essay","theological discourse",
                "memorial","consolatio","religious tract","oration and sermon","hymns",
                "vindication","scripture","criticism","observation","mock sermon",
                "biblical texts harmonized","theological tract","homiletic tract",
                "ecclesiastical tract","biblical text"]
    wanted_types = {k:None for k in wanted_types}

    unwanted = []
    def has_wanted_type(tag):
        if tag.get("TYPE") in wanted_types: 
            return True 
        # print(tag.get("TYPE"))
        unwanted.append(tag.get("TYPE"))
        return False 

    contents = [tag for tag in contents if not has_wanted_type(tag)]
    return unwanted 
    # sermon_text = []
    # # need to place delimiters between each sermon, marginal note, and page
    # p_idx = 0 
    # for idx, sermon in enumerate(contents):
    #     n_idx = 0 # note's relative position within sermon
    #     for t in sermon.find_all(['PB']):
    #         # page numbers 
    #         if t.name == "PB" and "N" in t.attrs: 
    #             page = t["N"]
    #             t.string = f' PAGE{page} '
    #         else: 
    #             page = t["REF"]
    #             t.string = f' PAGEIMAGE{page} '

        # for t in sermon.find_all(['P']):
        #     # show display characters for GAP elements (illegible characters, non-Latin alphabet) 
        #     for gap in t.find_all("GAP"):
        #         if gap["DESC"] == "foreign": 
        #             gap.string = " NONLATINALPHABET " # 〈 in non-Latin alphabet 〉
        #         elif "DISP" in gap.attrs: 
        #             disp = gap["DISP"]
        #             disp = re.sub("•","^",disp) # illegible letters 
        #             disp = re.sub("◊","*",disp) # illegible words 
        #             gap.string = disp
        #     for italics in t.find_all("HI"):
        #         italics.string = f" STARTITALICS {italics.text} ENDITALICS "
        #     for item in t.find_all(["NOTE"]):
        #         if item.name == "NOTE":
        #             # add note delimiters 
        #             item.string = f" STARTNOTE{n_idx} {item.text} ENDNOTE{n_idx} "
        #             n_idx += 1 
        #     t.string = f" STARTPARAGRAPH{p_idx} {t.text} "
        #     p_idx += 1 
    #     text = re.sub(r"[^\x00-\x7F]","",str(sermon.text))
    #     text = re.sub(r"[\{\}\[\]]","",text)
    #     text = f"{f' SERMON{idx}'} {text}"
    #     text = re.sub(r"\s+"," ",text)
    #     sermon_text.append(text)
    # with open(f"../assets/plain/{tcpID}.txt","w+") as file:
    #     file.writelines(" ".join(sermon_text)) # write as one long string          
    # return len(contents)

In [41]:
unwanted = []
from tqdm import tqdm 
progress_bar = tqdm(sorted(tcpIDs))
for tcpID in progress_bar:
    progress_bar.set_description(tcpID) 
    path = findTextTCP(tcpID)
    # print(path)
    u = extract(tcpID, path)
    unwanted.extend(u)

100%|██████████| 5858/5858 [10:26<00:00,  9.35it/s]


In [54]:
from collections import Counter
# print(sorted(Counter(unwanted).items(), key = lambda x:x[1], reverse=True))
unwanted_set = set(unwanted)
remaining = []
for tag_type in unwanted_set: 
    if re.search("sermon|speech|treatise|doctrine|comment|reasons|thesis|tenets|exposition",tag_type.lower()):
        # print(tag_type)
        continue
    else: 
        remaining.append(tag_type)
print(unwanted_set)

{'prefatory poem', 'response', 'chronology', 'Psalm', 'catalogue', "author's dedication", 'praises', 'elegies and epitaphs', 'blessing', "author's letter", 'hymn', 'table of contents for first treatise', 'sermons on Thessalonians', 'biography', 'diary excerpt', 'examination', 'list of heads', 'emblem of Westminster College', 'note on the text', 'response to allegations', 'remarks', 'author to the reader', 'sermons on Luke', 'rules of argument', 'sonnet', 'history of the return', 'list of cited authors', 'authorities', 'modern addenda', 'list of authors cited', 'vindication of author', 'anagrams', 'sermon on the Nativity', 'epigraph to the reader', 'royal proclamation', 'reasons', 'sermon preached at Wells', 'anthem', 'Pater Noster', 'council of Constance', 'prayers and meditations', 'imprimaturs', 'religious tracts', 'elegy (translation)', 'to the parishioner', 'religious treatise', 'index of prohibited books', 'answer to discourse', 'dramatic conversion', 'declaration', 'preamble', 'i

In [116]:

non_body = ["imprimatur","proclamation","council","decree","writ","license",
            "authorization","mandate","order","statute","publisher","advert",
            "dedication","list","index","outline","sentence",
            "question","table","coat","image","order","apology","dedication",
            "title","fealty","oath","preface","articles","errata",
            "declaration","bookplate","analysis of contents","summary of contents",
            "letter","frontispiece","summary","prayer","parallel histories",
            "appendix","life","biography","illustration","commission",
            "notice","epitaph","chronology","catalogue","diary","emblem",
            "sonnet","blessing","hymn","history","anthem","pater noster",
            "anagrams","modern addenda","drama","preamble","indictment","metrical",
            "printer","addendum","musical composition","summons","privilege","grant",
            "chart","will","parable","almanac","play","stories","story","seal",
            "edict","treaty","map","consent","record","creed","memorandum",
            "mantissa","inscription","translation of document","act of",
            "papal bull","formal","fabliau","citations","reference","bill",
            "label","to the",'author',"answer","song","poem"]
for t in set(unwanted): 
    if not re.search("|".join(non_body),t.lower()): 
        if not re.search("sermon|to the|poem|religious|elegy|commentary",t.lower()):
            print(t)

response
Psalm
praises
examination
note on the text
response to allegations
remarks
rules of argument
reasons
excerpts concerning excommunication
warning
thanksgivings
metaphrase
to Catholic readers
commandments
refutation
delivery date
Canticles
positions
moral treatise
Directions
testimony
dispute
prophecy
plan of temple
supplication
decalogue
epigram
responses
collection
thesis
tenets
dialogue
epistles
eulogies
exclamatory address
report
duplicate pages
proposals
epilogue
biblical paraphrase
addenda
epistle
judgment
exposition of Psalm 1
request to print
book excerpt
prefatory address
appeal
biblical source text
petition
resolution
demonstration of plagiarism
meditation
volume
propositions
appointment
account of portentous events
note
biblical admonition
censure
excerpt
paper
collection of lectures
sacraments
addition
ecclesiastical laws
theological tract
biblical texts
continuation
certificates
confession
contents
diagram
preacher to congregation
ode
biblical texts harmonized
poet


In [17]:
# remove false positives
plain_folder = "../assets/plain_all" 
from tqdm import tqdm
import os,re 
body_folder = {fp:None for fp in os.listdir("../assets/plain_body")}
none = []
excluded_div = []
other_div = []

for fp in tqdm(os.listdir(plain_folder)):
    tcpID = fp.split(".")[0]
    with open("{}/{}".format(plain_folder,fp)) as file: 
        data = file.readlines()[0]
    # if len(data) == 0: 
    #     print(tcpID)
    #     none.append(tcpID)
    # if re.search(r"SECTION\d+:articles",data): 
    #     none.append(tcpID)
    titles = re.search(r"SECTION\d+:(lecture|memorial|chapter|errata|typological_category|consolatio|section|biblical_commentary)",data)
    if titles is not None: 
        none.append(tcpID)
    # for t in titles: 
    #     if int(t) > 0: 
    #         none.append(tcpID)
    # divs = re.findall(r"SECTION\d+:([a-z\_]+)",data)
    # if fp not in body_folder: 
    #     print(tcpID, divs)
    #     # none.append(tcpID)
    #     # other_div.extend(divs)
    # else: 
    #     excluded_div.extend(divs)

        # os.remove("{}/{}".format(adorn_folder,fp))
        # print("removed {}".format(fp))
    # if tcpID not in tcpIDs: 
    #     os.remove("{}/{}".format(adorn_folder,fp))
    #     print("removed {}".format(fp))
# lecture, memorial, chapter, errata,
# typological_category, consolatio,section,biblical_commentary 
 

  0%|          | 0/5862 [00:00<?, ?it/s]

100%|██████████| 5862/5862 [00:08<00:00, 726.04it/s]


In [18]:
len(none)

1012

In [ ]:
for fp in os.listdir("../assets/adorned"): 
    tcpID = fp.split(".")[0]
    if tcpID in none: 
        os.remove(f"../assets/adorned/{fp}")
        print(f"Removed {fp}")

In [20]:
print(none)

['A90514', 'A74686', 'A59072', 'A90266', 'A80049', 'A56369', 'A85657', 'A29218', 'A70812', 'A71259', 'A62543', 'A61710', 'A97125', 'A90689', 'A62379', 'A38705', 'A65628', 'A30167', 'A47489', 'A36270', 'A61300', 'A80854', 'A21042', 'A11802', 'A91743', 'A14985', 'A48778', 'A04214', 'A76457', 'A21269', 'A56744', 'A42057', 'A03779', 'A65563', 'A61711', 'A30238', 'A67406', 'A40091', 'A47098', 'A72989', 'A90059', 'A07457', 'A06749', 'A81140', 'A45658', 'A63847', 'A46806', 'A25423', 'B15202', 'A58345', 'A03427', 'A78903', 'A42445', 'A67821', 'A03343', 'A58347', 'A30945', 'A36449', 'A59930', 'A50243', 'A90701', 'A44544', 'A00448', 'A58810', 'A36887', 'A31656', 'A63676', 'B14334', 'A77788', 'A42733', 'A00514', 'A60586', 'A97126', 'A93340', 'A91543', 'A42096', 'A35578', 'A61129', 'A77987', 'A15864', 'A20161', 'A49464', 'A60948', 'A59893', 'A01346', 'A69777', 'A52087', 'A52905', 'A67000', 'A44801', 'A48949', 'A16834', 'A80842', 'A61470', 'B12340', 'A18831', 'A81235', 'A14992', 'A52051', 'A46742',

In [2]:
excluded_div, other_div = set(excluded_div),set(other_div)
# other_div = [x for x in other_div if x not in excluded_div]

In [3]:
other_div

{'account',
 'account_of_portentous_events',
 'address',
 'answer',
 'answer_to_preface',
 'appendix',
 'argument',
 'author',
 'biblical_commentary',
 'bibliography',
 'biography',
 'canticles',
 'catechism',
 'chapter',
 'class',
 'collection',
 'colophon',
 'consolatio',
 'contents',
 'decade',
 'dedication',
 'dialogue',
 'document',
 'elegy',
 'embassage',
 'epigraph',
 'epitaph',
 'errata',
 'frontispiece',
 'half_title',
 'hymns',
 'illustration',
 'imprimatur',
 'index',
 'index_of_psalms',
 'introduction',
 'intrusive_page',
 'jacobite_articles_of_faith',
 'lecture',
 'letter',
 'license',
 'list_of_scriptural_passages',
 'list_of_songs',
 'liturgical_season',
 'memorial',
 'metrical_psalms',
 'narrative',
 'note',
 'notice',
 'order',
 'parliamentary_declaration',
 'petition',
 'poem',
 'poems',
 'postscript',
 'prayer',
 'prayers',
 'preface',
 'prefatory_letter',
 'printer_to_the_reader',
 'psalms',
 'publisher',
 'quotation',
 'reasons',
 'references',
 'royal_license',
 '

In [119]:
for file in none: 
    os.remove(f"{plain_folder}/{file}.txt")

In [ ]:
# remove duplicates 
for file in os.listdir(adorn_folder): 
    if "-001" in file: 
        os.remove("{}/{}".format(adorn_folder,file))
        print("removed {}".format(file))